In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

In [2]:
# generating dataset for discriminator
def get_dataset():
    return torch.Tensor(np.random.normal(4,1.25,(1,100)))

In [3]:
# generating noise for generator
def make_noise():
    return torch.Tensor(np.random.uniform(0,0,(1,50)))

In [4]:
class generator(nn.Module):
    
    def __init__(self,inp,out):
        super(generator,self).__init__()
        self.net = nn.Sequential(nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                nn.Linear(300,300),
                                 nn.ReLU(inplace=True),
                                nn.Linear(300,out)
                               )
        
    def forward(self,x):
        x = self.net(x)
        return x
        

In [5]:
class discriminator(nn.Module):
    
    def __init__(self,inp,out):
        super(discriminator,self).__init__()
        self.net = nn.Sequential(nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,out),
                                 nn.Sigmoid()
                                )
        
    def forward(self,x):
        x = self.net(x)
        return x

In [6]:
def stats(array):
    array = array.detach().numpy()
    return [np.mean(array),np.std(array)]

In [7]:
gen = generator(50,100)

In [8]:
gen

generator(
  (net): Sequential(
    (0): Linear(in_features=50, out_features=300, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=300, out_features=100, bias=True)
  )
)

In [9]:
x = make_noise()

In [10]:
x1 = gen(x)

In [11]:
stats(x1)

[0.0018841184, 0.036860824]

In [12]:
discrim = discriminator(100,1)

In [13]:
discrim

discriminator(
  (net): Sequential(
    (0): Linear(in_features=100, out_features=300, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=300, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [14]:
x2 = discrim(x1)

In [15]:
x2

tensor([[ 0.5205]])

In [16]:
epochs = 500

In [17]:

d_step = 10
g_step = 8
criteriond1 = nn.BCELoss()
optimizerd1 = optim.SGD(discrim.parameters(), lr=0.001, momentum=0.9)

criteriond2 = nn.BCELoss()
optimizerd2 = optim.SGD(gen.parameters(), lr=0.001, momentum=0.9)

printing_steps = 20

In [18]:
for epoch in range(epochs):
    
    if epoch%printing_steps==0:
        print("Epoch:", epoch)
    
    # training discriminator
    for d_i in range(d_step):
        discrim.zero_grad()
        
        #real
        data_d_real = Variable(get_dataset())
        data_d_real_pred = discrim(data_d_real)
        data_d_real_loss = criteriond1(data_d_real_pred,Variable(torch.ones(1,1)))
        data_d_real_loss.backward()
        
        
        #fake
        data_d_noise = Variable(make_noise())
        data_d_gen_out = gen(data_d_noise).detach()
        data_fake_dicrim_out = discrim(data_d_gen_out)
        data_fake_d_loss = criteriond1(data_fake_dicrim_out,Variable(torch.zeros(1,1)))
        data_fake_d_loss.backward()
        
        optimizerd1.step()
        
    for g_i in range(g_step):
        
        gen.zero_grad()
        
        data_noise_gen = Variable(make_noise())
        data_g_gen_out = gen(data_noise_gen)
        data_g_dis_out = discrim(data_g_gen_out)
        data_g_loss = criteriond2(data_g_dis_out,Variable(torch.ones(1,1)))
        data_g_loss.backward()
        
        optimizerd2.step()
        
        if epoch%printing_steps==0:
            print(stats(data_g_gen_out))
    
    if epoch%printing_steps==0:
        print("\n\n")
    

('Epoch:', 0)
[0.0018841184, 0.036860824]
[0.001885428, 0.036860917]
[0.0018879148, 0.03686111]
[0.0018914625, 0.036861386]
[0.0018959647, 0.036861762]
[0.0019013253, 0.036862247]
[0.0019074584, 0.03686284]
[0.0019142872, 0.036863554]



('Epoch:', 20)
[0.007975619, 0.038056422]
[0.008030179, 0.03808928]
[0.008086704, 0.03812305]
[0.008145271, 0.03815762]
[0.008205742, 0.038193222]
[0.008268582, 0.038229816]
[0.008333553, 0.038267855]
[0.008401263, 0.038307115]



('Epoch:', 40)
[0.025739962, 0.07039434]
[0.025967699, 0.07082409]
[0.026195178, 0.071255475]
[0.026425453, 0.071689904]
[0.026660535, 0.072128154]
[0.026900362, 0.07257016]
[0.027144287, 0.07301512]
[0.027392678, 0.07346259]



('Epoch:', 60)
[0.08450457, 0.14448352]
[0.08509155, 0.14511523]
[0.08568327, 0.14574906]
[0.08627997, 0.14638585]
[0.0868816, 0.14702617]
[0.08748844, 0.1476706]
[0.08809981, 0.1483197]
[0.08871749, 0.1489742]



('Epoch:', 80)
[0.28840774, 0.27236685]
[0.29072142, 0.2731846]
[0.29304278, 0.27398548]

In [19]:
gen

generator(
  (net): Sequential(
    (0): Linear(in_features=50, out_features=300, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): ReLU(inplace)
    (4): Linear(in_features=300, out_features=100, bias=True)
  )
)

In [20]:
# torch.save(gen.state_dict(),"d_step_5000_g_step_2000")

In [21]:
y = gen(make_noise())

In [22]:
stats(y)

[5.0801945, 0.45474356]